# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

An equal-weight version of an index fund is a type of index fund where each constituent stock is given the same importance or weight in the portfolio, regardless of the company's market capitalization. This contrasts with traditional market-capitalization-weighted index funds, where the weighting of each stock in the index is proportional to its market value, meaning larger companies have a greater influence on the index's performance.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [136]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [137]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [138]:
from tokenz import IEX_CLOUD_API_TOKEN

IEX_CLOUD_API_TOKEN

'sk_850f6a1b41c94a3baf2e08a60022252a'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [139]:
symbol='aapl'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token='
url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# url
# url = 'https://api.iex.cloud/v1/data/core/quote/aapl?token=sk_850f6a1b41c94a3baf2e08a60022252a'
outer_data = requests.get(url).json()
outer_data





[{'avgTotalVolume': 65017399,
  'calculationPrice': 'tops',
  'change': 1.42,
  'changePercent': 0.00762,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': None,
  'highTime': None,
  'iexAskPrice': 188,
  'iexAskSize': 220,
  'iexBidPrice': 187.69,
  'iexBidSize': 289,
  'iexClose': 187.7,
  'iexCloseTime': 1715714920736,
  'iexLastUpdated': 1715714920736,
  'iexMarketPercent': 0.011025418558441191,
  'iexOpen': 187.64,
  'iexOpenTime': 1715693400488,
  'iexRealtimePrice': 187.7,
  'iexRealtimeSize': 1,
  'iexVolume': 403823,
  'lastTradeTime': 1715714920736,
  'latestPrice': 187.7,
  'latestSource': 'IEX real time price',
  'latestTime': '3:28:40 PM',
  'latestUpdate': 1715714920736,
  'latestVolume': None,
  'low': None,
 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [140]:
data = outer_data[0]
price = data['latestPrice']
market_cap = data['marketCap']


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [141]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [142]:
new_row = pd.Series(['AAPL', data['latestPrice'], data['marketCap'], 'N/A'], index = my_columns)
final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
final_dataframe

C:\Users\Aryan\AppData\Local\Temp\ipykernel_13336\2778207613.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,187.7,2878207191400,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [147]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker'][:50]:
    # print(symbol)
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    outer_data = requests.get(api_url).json()
    data = outer_data[0]
    new_row = pd.Series([symbol, data['latestPrice'], data['marketCap'], 'N/A'], index = my_columns)
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)


In [146]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,150.990,44248417331,N/A
1,AAL,15.150,9939768712,N/A
2,AAP,75.460,4498130682,N/A
3,AAPL,187.740,2878820554680,N/A
4,ABBV,161.185,284631398280,N/A
5,ABC,179.980,36387430583,N/A
6,ABMD,381.020,17180642928,N/A
7,ABT,103.715,180426115315,N/A
8,ACN,307.420,206101051926,N/A
9,ADBE,476.990,213691520000,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [86]:
# # Function sourced from 
# # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]

In [87]:
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [ ]:
# batch_api_url = f'https://apis.iex.cloud/v1/data/core/QUOTE,FUNDAMENTALS,NEWS/{symbol}?last=2&token={IEX_CLOUD_API_TOKEN}'
# symbol = 'A'
# batch_api_url = f'https://apis.iex.cloud/v1/data/core/QUOTE,FUNDAMENTALS,NEWS/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# # batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
# outer_data = requests.get(batch_api_url).json()
# outer_data[0]


In [ ]:
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
# #     print(symbol_strings[i])

# final_dataframe = pd.DataFrame(columns = my_columns)

# for symbol_string in symbol_strings:
# #     print(symbol_strings)
#     # batch_api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
#     batch_api_url = f'https://apis.iex.cloud/v1/data/core/QUOTE,FUNDAMENTALS,NEWS/{symbol_string}?last=2&token={IEX_CLOUD_API_TOKEN}'
#     # batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#     outer_data = requests.get(batch_api_url).json()
#     data = outer_data[0]
    
#     for symbol in symbol_string.split(','):
#         new_row = pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['marketCap'], 'N/A'], index = my_columns)
#         final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
        
    
# final_dataframe

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [151]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [152]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,151,44251347884,1
1,AAL,15.15,9939768712,13
2,AAP,75.52,4501707250,2
3,AAPL,187.82,2880047281240,1
4,ABBV,161.195,284649056958,1
5,ABC,179.98,36387430583,1
6,ABMD,381.02,17180642928,0
7,ABT,103.66,180330435458,1
8,ACN,307.43,206107756143,0
9,ADBE,476.94,213669120000,0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [153]:
writer = pd.ExcelWriter('recommend_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [155]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [156]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [157]:
writer.save()

AttributeError: 'XlsxWriter' object has no attribute 'save'